In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# For IBM watsonx on IBM Cloud

import getpass

# cloud_apikey = getpass.getpass("Type your IBM Cloud API key: ")
cloud_apikey = "1C_e21y5BVpQrTQjXgZCyXGkwt27NgsAlhWS2BmRrC-5"

url = "https://us-south.ml.cloud.ibm.com"

In [6]:
credentials = { 
    "url"    : url, 
    "apikey" : cloud_apikey
}

In [8]:
loader = PyPDFLoader("Knowlege_Base_for_LLM.pdf")
pages = loader.load_and_split()

In [10]:
# Split
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\nzblithereplz\n"],
    chunk_size=1500,
    chunk_overlap=250,
    length_function=len,
    is_separator_regex=False,
)
all_splits = text_splitter.split_documents(pages)
print(all_splits[1].page_content.replace("\n", " "))
embedding = GPT4AllEmbeddings()

zblithereplz ● Developing Secure Software - Learn the security basics to develop software that is hardened against attacks, and understand how you can reduce the damage and speed the response when a vulnerability is exploited. Thanks to the involvement of OpenSSF , a cross-industry collaboration that brings together leaders to improve the security of open source software by


Downloading: 100%|██████████| 45.9M/45.9M [00:05<00:00, 9.14MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 297MiB/s]


In [12]:
# Index
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=embedding,
)
retriever = vectorstore.as_retriever()

In [13]:
search_term = "advanced quantum computing"
documents = retriever.get_relevant_documents(search_term)
for doc in retriever.get_relevant_documents(search_term):
    doc.page_content = doc.page_content.replace("\n", " ").replace("zblithereplz", "").replace("-", "").replace("—", "")
    # print(doc.page_content[:1000] + "...")
    # print(len(doc.page_content))
article_txt = max(documents, key=lambda doc: len(doc.page_content)).page_content.replace("\n", " ").replace("zblithereplz", "").replace("-", "").replace("—", "")

/Users/a747/ai_grp/RagBot/env/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [14]:
prompt_template = """
Article:
###
%s
###

You are a chatbot that recommends courses to students. You are provided with an article that contains information about various courses.
Based on the users input you will recommend a course to the user.
Answer in a complete sentence, with proper capitalization and punctuation. 
If there is no good answer in the article, say "Sorry, I don't know".
Always include the course name in your answer.
Always include the course link in your answer.

User input: %s
Answer and link: 
"""

def augment( template_in, context_in, query_in ):
    return template_in % ( context_in,  query_in )

In [34]:
query = "I've done some work in quantum computing. What courses would you recommend?"

augmented_prompt = augment( prompt_template, article_txt, query)

In [39]:
import os

from ibm_watson_machine_learning.foundation_models import Model

model_id = "google/flan-t5-xxl"

gen_parms = { 
    "DECODING_METHOD" : "greedy", 
    "MIN_NEW_TOKENS" : 1, 
    "MAX_NEW_TOKENS" : 200 
}

# project_id = os.environ["PROJECT_ID"]
project_id = "9e074867-9d55-49d5-87d1-f0f696be8d84"


model = Model( model_id, credentials, gen_parms, project_id )


Error getting IAM Token.
Reason: <Response [400]>


WMLClientError: Error getting IAM Token.
Reason: <Response [400]>

In [31]:
import json

def generate( model_in, augmented_prompt_in ):
    
    generated_response = model_in.generate( augmented_prompt_in )

    if ( "results" in generated_response ) \
       and ( len( generated_response["results"] ) > 0 ) \
       and ( "generated_text" in generated_response["results"][0] ):
        return generated_response["results"][0]["generated_text"]
    else:
        print( "The model failed to generate an answer" )
        print( "\nDebug info:\n" + json.dumps( generated_response, indent=3 ) )
        return ""

In [32]:
output = generate( model, augmented_prompt )
print( output )

NameError: name 'model' is not defined

In [26]:
def searchAndAnswer( knowledge_base_in, model ):
    
    question = input( "Type your question:\n")
    if not re.match( r"\S+", question ):
        print( "No question")
        return
        
    # Retrieve the relevant content
    top_matching_index = search( question, knowledge_base_in )
    if top_matching_index < 0:
        print( "No good answer was found in the knowledge base" )
        return;
    asset = knowledge_base_in[top_matching_index]
    asset_txt = asset["txt"]
    
    # Augment a prompt with context
    augmented_prompt = augment( prompt_template, asset_txt, question )
    
    # Generate output
    output = generate( model, augmented_prompt )
    if not re.match( r"\S+", output ):
        print( "The model failed to generate an answer")
    print( "\nAnswer:\n" + output )
    print( "\nSource: \"" + asset["title"] + "\", " + asset["Author"] + " (" + asset["Published"] + ")"  )

In [ ]:
# searchAndAnswer( knowledge_base, model )